## Affectus analysis


Install the required libraries


In [1]:
! pip install --upgrade setuptools wheel
! pip install kaggle
! pip install numpy
! pip install pandas
! pip install wordcloud
! pip install matplotlib
! pip install seaborn
! pip install scikit-learn
! pip install xgboost
! pip install scikit-learn
! pip install nltk


APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL=1
APP_DEBUG=true
APP_ENV=local
APP_HOST=127.0.0.1
APP_KEY=mTO7A4wTQC2NRz76UtSB2vmosVBjMv2g
APP_NAME='Affectus Analysis'
APP_PORT=3000
APP_URL=http://localhost
CLICOLOR=1
CLICOLOR_FORCE=1
COMMAND_MODE=unix2003
DB_CONNECTION=mysql
DB_DATABASE=affectus-analysis
DB_HOST=127.0.0.1
DB_PASSWORD=''
DB_PORT=3306
DB_USERNAME=root
ELECTRON_NO_ATTACH_CONSOLE=1
ELECTRON_RUN_AS_NODE=1
FORCE_COLOR=1
GIT_PAGER=cat
GO111MODULE=on
GOPATH=/Users/arfanxn/Developments/Golang/
GRPC_HOST=''
GRPC_PORT=''
HOME=/Users/arfanxn
HOMEBREW_CELLAR=/opt/homebrew/Cellar
HOMEBREW_PREFIX=/opt/homebrew
HOMEBREW_REPOSITORY=/opt/homebrew
INFOPATH=/opt/homebrew/share/info:
JWT_SECRET=6kKEQ1ZXYlCo18odfg8m7NXZDyH50f8T
LC_CTYPE=UTF-8
LESS=-R
LOGNAME=arfanxn
LOG_CHANNEL=stack
LOG_LEVEL=debug
LSCOLORS=Gxfxcxdxbxegedabagacad
LS_COLORS='di=1;36:ln=35:so=32:pi=33:ex=31:bd=34;46:cd=34;43:su=30;41:sg=30;46:tw=30;42:ow=30;43'
LaunchInstanceID=4998763A-F416-47B3-AB4A-5BF4FE1F51CE
MAIL_FROM_ADDR

Import the dependencies


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing
pd.options.mode.chained_assignment = None
import os #File location
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

from wordcloud import WordCloud #Word visualization
import matplotlib.pyplot as plt #Plotting properties
import seaborn as sns #Plotting properties
from sklearn.feature_extraction.text import CountVectorizer #Data transformation
from sklearn.model_selection import train_test_split #Data testing
from sklearn.linear_model import LogisticRegression #Prediction Model
from sklearn.metrics import accuracy_score #Comparison between real and predicted
from xgboost import XGBClassifier
from sklearn.preprocessing import LabelEncoder #Variable encoding and decoding for XGBoost
import re #Regular expressions
import nltk
import pickle
from nltk import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/arfanxn/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

Configurations


In [3]:
# ! kaggle config set username <your_kaggle_username>
# ! kaggle config set key <your_kaggle_key>

dataset_author = 'jp797498e'
dataset_title = 'twitter-entity-sentiment-analysis'
dataset_column_names = ['tweet_id', 'entity', 'sentiment', 'text'] 

kaggle_path = './resources/kaggle'

train_dataset_csv_filename = 'twitter_training.csv'
validation_dataset_csv_filename = 'twitter_validation.csv'

model_pickle_filename = 'affectus-analysis-model.pickle'
vectorizer_pickle_filename = 'affectus-analysis-vectorizer.pickle'


Download dataset from Kaggle


In [4]:
# download dataset from kaggle api
! kaggle datasets download -d {dataset_author+'/'+dataset_title} -p {kaggle_path} 

APPLICATION_INSIGHTS_NO_DIAGNOSTIC_CHANNEL=1
APP_DEBUG=true
APP_ENV=local
APP_HOST=127.0.0.1
APP_KEY=mTO7A4wTQC2NRz76UtSB2vmosVBjMv2g
APP_NAME='Affectus Analysis'
APP_PORT=3000
APP_URL=http://localhost
CLICOLOR=1
CLICOLOR_FORCE=1
COMMAND_MODE=unix2003
DB_CONNECTION=mysql
DB_DATABASE=affectus-analysis
DB_HOST=127.0.0.1
DB_PASSWORD=''
DB_PORT=3306
DB_USERNAME=root
ELECTRON_NO_ATTACH_CONSOLE=1
ELECTRON_RUN_AS_NODE=1
FORCE_COLOR=1
GIT_PAGER=cat
GO111MODULE=on
GOPATH=/Users/arfanxn/Developments/Golang/
GRPC_HOST=''
GRPC_PORT=''
HOME=/Users/arfanxn
HOMEBREW_CELLAR=/opt/homebrew/Cellar
HOMEBREW_PREFIX=/opt/homebrew
HOMEBREW_REPOSITORY=/opt/homebrew
INFOPATH=/opt/homebrew/share/info:
JWT_SECRET=6kKEQ1ZXYlCo18odfg8m7NXZDyH50f8T
KMP_DUPLICATE_LIB_OK=True
KMP_INIT_AT_FORK=FALSE
LC_CTYPE=UTF-8
LESS=-R
LOGNAME=arfanxn
LOG_CHANNEL=stack
LOG_LEVEL=debug
LSCOLORS=Gxfxcxdxbxegedabagacad
LS_COLORS='di=1;36:ln=35:so=32:pi=33:ex=31:bd=34;46:cd=34;43:su=30;41:sg=30;46:tw=30;42:ow=30;43'
LaunchInstanceID=49

Extract the downloaded dataset(s)


In [5]:
from zipfile import ZipFile

with ZipFile(kaggle_path+'/'+dataset_title+'.zip', 'r') as zip:
    zip.extractall(kaggle_path)

Read datasets


In [6]:
# Training dataset
trainDf = pd.read_csv(kaggle_path + '/' +train_dataset_csv_filename, header=None) 

# Validation dataset
validationDf = pd.read_csv(kaggle_path + '/' +validation_dataset_csv_filename, header=None) 

Set train dataset column names then show it


In [7]:
trainDf.columns= dataset_column_names
trainDf.head()

,tweet_id,entity,sentiment,text
0,2401,Borderlands,Positive,im getting on borderlands and i will murder yo...
1,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
2,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
3,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
4,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...


Set validation dataset column names then show it


In [8]:
validationDf.columns= dataset_column_names
validationDf.head()

,tweet_id,entity,sentiment,text
0,3364,Facebook,Irrelevant,I mentioned on Facebook that I was struggling ...
1,352,Amazon,Neutral,BBC News - Amazon boss Jeff Bezos rejects clai...
2,8312,Microsoft,Negative,@Microsoft Why do I pay for WORD when it funct...
3,4371,CS-GO,Negative,"CSGO matchmaking is so full of closet hacking,..."
4,4433,Google,Neutral,Now the President is slapping Americans in the...


To prepare the data for the text analysis an additional row was created using the method of `str.transformed_text`. However, as there were some texts with only numerical values (such as one that only had a 2 as the tweet) an additional function was used for transforming all the data to string.

Then, a regex expression erased the special characters as it is common to have digitation problems on Twitter.


In [9]:
#Text transformation
trainDf["transformed_text"]=trainDf.text.str.lower() #lowercase
trainDf["transformed_text"]=[str(data) for data in trainDf.transformed_text] #converting all to string
trainDf["transformed_text"]=trainDf.transformed_text.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x)) #regex
validationDf["transformed_text"]=validationDf.text.str.lower() #lowercase
validationDf["transformed_text"]=[str(data) for data in validationDf.transformed_text] #converting all to string
validationDf["transformed_text"]=validationDf.transformed_text.apply(lambda x: re.sub('[^A-Za-z0-9 ]+', ' ', x)) #regex

The differences between the two text columns are presented in the next table.


In [10]:
trainDf.tail()

,tweet_id,entity,sentiment,text,transformed_text
74677,9200,Nvidia,Positive,Just realized that the Windows partition of my...,just realized that the windows partition of my...
74678,9200,Nvidia,Positive,Just realized that my Mac window partition is ...,just realized that my mac window partition is ...
74679,9200,Nvidia,Positive,Just realized the windows partition of my Mac ...,just realized the windows partition of my mac ...
74680,9200,Nvidia,Positive,Just realized between the windows partition of...,just realized between the windows partition of...
74681,9200,Nvidia,Positive,Just like the windows partition of my Mac is l...,just like the windows partition of my mac is l...


Initialize vectorizer and vectorizes


In [11]:
#n-gram of 4 words
vectorizer = CountVectorizer(
    tokenizer=word_tokenize,
    ngram_range=(1,4)
)

#Train - Test splitting
X_train, X_test = train_test_split(trainDf, test_size=0.2, random_state=0)

#Data labeling
X_train_vectorized = vectorizer.fit_transform(X_train.transformed_text)
X_test_vectorized = vectorizer.transform(X_test.transformed_text)
X_validation_vectorized = vectorizer.transform(validationDf.transformed_text)

y_train_vectorized = X_train['sentiment']
y_test_vectorized = X_test['sentiment']

/Users/arfanxn/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:523: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Train the LogisticRegression model


In [12]:
model = LogisticRegression(C=0.9, solver="liblinear",max_iter=1500)
model.fit(X_train_vectorized, y_train_vectorized)

Predict with the previously trained model


In [ ]:
# Prediction
predictions = model.predict(X_test_vectorized)
print("Accuracy: ", accuracy_score(y_test_vectorized, predictions) * 100)

Accuracy:  90.79467095132891
Shape of X_test_vectorized: (14937, 1427378)
Shape of y_test_vectorized: (14937,)


Predict with validation data


In [ ]:
y_validation_vectorized = validationDf['sentiment']
predictions = model.predict(X_validation_vectorized)
print("Accuracy: ", accuracy_score(y_validation_vectorized, predictions) * 100)

Accuracy:  98.6


Save model and vectorizer with pickle


In [ ]:
# save with pickle
pickle.dump(model, open(kaggle_path + '/' + model_pickle_filename, 'wb'))

# save additional required parameters
pickle.dump(vectorizer, open(kaggle_path + "/" + vectorizer_pickle_filename, "wb"))

Use saved model and vectorizer


In [ ]:
model = pickle.load(open(kaggle_path+'/'+model_pickle_filename, 'rb'))
vectorizer = pickle.load(open(kaggle_path+'/'+vectorizer_pickle_filename, 'rb'))

sentiment_texts = [
        'I like this but sometimes i dont like it',
    ]
X_custom_vectorized = vectorizer.transform(sentiment_texts)
predictions = model.predict(X_custom_vectorized)
print('"'+sentiment_texts[0]+'" is a '+str(predictions[0]).lower()+' sentiment.')

print('Shape of X_custom_vectorized:',X_custom_vectorized.shape)
print('Shape of predictions:',predictions.shape)

"I like this but sometimes i dont like it" is a positive sentiment.
Shape of X_custom_vectorized: (1, 1427378)
Shape of predictions: (1,)
